In [12]:
import nltk
import pandas as pd
import sklearn
import re  
import os

import pyLDAvis.sklearn as LDAvis
import pyLDAvis

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
## For Stemming


from sklearn.decomposition import NMF, LatentDirichletAllocation

import warnings
warnings.filterwarnings('ignore')

In [13]:
# functions
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])

In [14]:
#csv_file = 'tables_v2/v2_coach_search_result_tweet_text_data_unique.csv'
#csv_file = 'tables_v2/v2_nfl_tweets_master.csv'
#csv_file = 'tables_v2/v2_player_search_result_tweet_text_data_unique.csv'
#csv_file = 'tables_v2/v2_team_search_result_tweet_text_data_unique.csv'


#csv_file = 'tables/coach_search_result_tweet_text_data_unique.csv'
csv_file = 'tables/nfl_tweets_master.csv'
#csv_file = 'tables/player_search_result_tweet_text_data_unique.csv'
#csv_file = 'tables/team_search_result_tweet_text_data_unique.csv'

df_coaches = pd.read_csv(csv_file)

In [15]:
df_coaches.head()

,text
0,You mean the same fan experts who could see La...
1,Deshaun Watson or Sam Darnold? WhoShouldIStart
2,RT Top 5 current NFL quarterbacks 1. Tom Bra...
3,A NFL team I love: Browns A NFL team I like: P...
4,Deshaun Watson


In [16]:
my_list=[] # list to store results
shortword = re.compile(r'\W*\b\w{1,1}\b') # filtering out all words with len <= 1
my_list = [shortword.sub('', df_coaches.iloc[i,0]) for i in range(len(df_coaches))]

In [17]:
no_features = 1000
lda_cv = CountVectorizer(input="content", analyzer = 'word', stop_words = 'english', lowercase = True, 
                         max_df = 0.95, min_df = 2, max_features = no_features, encoding="ISO-8859-1")
cv = lda_cv.fit_transform(my_list)

In [18]:
CV = lda_cv.fit_transform(my_list)
lda_columns = lda_cv.get_feature_names()
df_corpus_lda = pd.DataFrame(CV.toarray(),columns = lda_columns)
df_corpus_lda.head()

,00,000,01,03,07s,10,100,103,10maillotsen10jours,11,...,wr,wylie,yahoo,yards,yds,year,years,yes,young,yr
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
no_topics = 5
max_iterations = 10
learn_off = 50
random = 0

lda_model = LatentDirichletAllocation(n_components = no_topics, max_iter = max_iterations, learning_method='online',
                                         learning_offset = learn_off,random_state = random)
#lda_model = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online')
LDA_DH_Model = lda_model.fit_transform(CV)

In [20]:
print(LDA_DH_Model.shape) # docs | topics
print(LDA_DH_Model[0])

(10089, 5)
[0.35564008 0.55821145 0.02873212 0.02871924 0.02869711]


In [21]:
print_topics(lda_model, lda_cv)

Topic 0:
[('watson', 1133.1372335116298), ('yr', 1132.5586796895684), ('deshaun', 1083.4415245305202), ('7m', 323.525537366837), ('game', 302.9915057956501), ('rt', 261.7479736646498), ('astros', 244.72134278460953), ('clemson', 244.55490788455012), ('season', 239.62478814822123), ('mahomes', 221.03740492759553)]
Topic 1:
[('deshaun', 1412.427528280317), ('watson', 1409.6878223628619), ('mahomes', 637.0957989409143), ('patrick', 518.2868189598968), ('qbs', 507.00505463195987), ('qb', 424.81987098857235), ('lamar', 407.2911039796762), ('jackson', 394.85530688272763), ('defense', 388.65597982026213), ('patriots', 380.6680927155648)]
Topic 2:
[('watson', 790.9981974278076), ('deshaun', 539.0893910205733), ('texans', 527.1940485655715), ('games', 478.7254300708557), ('work', 388.93511091855447), ('yards', 353.79418595272256), ('texas', 339.60983258353116), ('want', 338.40213801893856), ('drop', 312.5183769284415), ('fellow', 292.9246472220239)]
Topic 3:
[('texans', 1880.4971088959207), ('w

In [22]:
panel = LDAvis.prepare(lda_model, CV, lda_cv, mds='tsne')
'''
Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
'''
pyLDAvis.show(panel)


Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8891/    [Ctrl-C to exit]


127.0.0.1 - - [04/Dec/2019 18:20:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2019 18:20:09] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2019 18:20:09] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2019 18:20:10] "GET /LDAvis.js HTTP/1.1" 200 -



stopping Server...
